Data Processing and Separation
==============================

In [1]:
!pip install librosa

In [3]:
import pandas as pd
import numpy as np
import librosa as lib

In [22]:
valid_data = pd.read_csv("../valid_data.csv")

# we need phone position, affect, file_path, then the local file path
valid_data["file_path"] = valid_data["file_name"].str.replace("/media/data/shout-data/", "/dataset/")
valid_data["file_path"] = valid_data["file_path"].str.replace(".wav", "")
valid_data["file_path"] = valid_data["file_path"].astype(str) + valid_data["chunk_name"].astype(str)

processingSet = valid_data[["affect", "file_path", "phone_position"]]

testSet = valid_data.sample(n=100, random_state=100)
testSet.to_csv("../test_set.csv")

trainingSet = valid_data.drop(testSet.index)
trainingSet.to_csv("../training_set.csv")

In [23]:
processingSet

,affect,file_path,phone_position
0,sadness,/dataset/shout_data_3afd7208-7987-4c73-8e01-58...,Place phone on the opposite side of the room i...
1,neutral,/dataset/shout_data_ae6feb35-cf7a-4805-8d05-03...,Hold your phone next to your face but with the...
2,neutral,/dataset/shout_data_fdb758a5-9c86-40f7-8870-ca...,Place phone on the opposite side of the room f...
3,neutral,/dataset/shout_data_fdb758a5-9c86-40f7-8870-ca...,Place phone on the opposite side of the room f...
4,disgust,/dataset/shout_data_739dbf1c-ec21-42db-9e49-3a...,Place phone 1-2 meters away face up on any sur...
...,...,...,...
3048,sadness,/dataset/shout_data_f8d1719f-8a4b-4d7a-a2a7-ef...,Place phone as far away as possible while on o...
3049,sadness,/dataset/shout_data_f8d1719f-8a4b-4d7a-a2a7-ef...,Place phone as far away as possible while on o...
3050,sadness,/dataset/shout_data_f8d1719f-8a4b-4d7a-a2a7-ef...,Place phone as far away as possible while on o...
3051,fear,/dataset/shout_data_d5d4e179-d411-4652-b80b-3b...,Place phone 1-2 meters away face down on a har...


Time Domain Feature Extraction
===================
https://www.youtube.com/watch?v=SRrQ_v-OOSg

https://www.youtube.com/watch?v=rlypsap6Wow

https://www.youtube.com/watch?v=EycaSbIRx-0

Time-domain features:
- Amplitude envelope 
- Root-mean-square energy 
- Zero-crossing rate


,age,current_language,first_language,gender,script,affect,phone_position,file_name,chunk_name,phone_model
0,25-39,English,English,Woman,another dog secretary show,sadness,Place phone on the opposite side of the room i...,/media/data/shout-data/shout_data_3afd7208-798...,/final/chunk42.wav,iPhone 12
1,40-60,English,Italian,Man,five special guest treasures,neutral,Hold your phone next to your face but with the...,/media/data/shout-data/shout_data_ae6feb35-cf7...,/final/chunk20.wav,iPhone 6
2,18-24,English,English,Woman,wolf ring lights are fantastic,neutral,Place phone on the opposite side of the room f...,/media/data/shout-data/shout_data_fdb758a5-9c8...,/final/chunk20.wav,Iphone
3,18-24,English,English,Woman,wolf ring lights are fantastic,neutral,Place phone on the opposite side of the room f...,/media/data/shout-data/shout_data_fdb758a5-9c8...,/final/chunk30.wav,Iphone
4,25-39,English,English,Woman,birds make new jingles,disgust,Place phone 1-2 meters away face up on any sur...,/media/data/shout-data/shout_data_739dbf1c-ec2...,/final/chunk11.wav,Samsung Galaxy s8+
...,...,...,...,...,...,...,...,...,...,...
3048,25-39,English,Spanish,Man,fluffy baseboard yogurt division,sadness,Place phone as far away as possible while on o...,/media/data/shout-data/shout_data_f8d1719f-8a4...,/final/chunk02.wav,iPhone
3049,25-39,English,Spanish,Man,fluffy baseboard yogurt division,sadness,Place phone as far away as possible while on o...,/media/data/shout-data/shout_data_f8d1719f-8a4...,/final/chunk00.wav,iPhone
3050,25-39,English,Spanish,Man,fluffy baseboard yogurt division,sadness,Place phone as far away as possible while on o...,/media/data/shout-data/shout_data_f8d1719f-8a4...,/final/chunk01.wav,iPhone
3051,60 and over,English,English,Woman,zebra cats walking on lamps,fear,Place phone 1-2 meters away face down on a har...,/media/data/shout-data/shout_data_d5d4e179-d41...,/final/chunk01.wav,Samsung Galaxy 8


Frequency Domain Feature Extraction
=========